#0) Imports and Installs

In [353]:
!pip install yfinance

In [354]:
!pip install numerize

In [355]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

import datetime as dt

# for graphs
import matplotlib.pyplot as plt

import math

#1) Question 1: IPO Filings Web Scraping and Data Processing

In [356]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings"
response = requests.get(url, headers=headers)

ipo_filings_html = pd.read_html(response.text)

In [357]:
ipo_filings_dfs = ipo_filings_html[0]
ipo_filings_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [358]:
ipo_filings_dfs.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [359]:
ipo_filings_dfs['Filing Date'] = pd.to_datetime(ipo_filings_dfs['Filing Date'], format='%b %d, %Y')
ipo_filings_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  325 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 12.8+ KB


In [360]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
ipo_filings_dfs['Shares Offered'] = pd.to_numeric(ipo_filings_dfs['Shares Offered'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
ipo_filings_dfs['Shares Offered'] = pd.to_numeric(ipo_filings_dfs['Shares Offered'])
ipo_filings_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  252 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [361]:
import re
nums_strings = re.findall(r"(?:[\£\$\€]{1}[,\d]+.?\d*)", "$3.00")
print(nums_strings)

from statistics import mean
nums_floats = [float(string.strip('$')) for string in nums_strings]
print(nums_floats)
print(round(mean(nums_floats), 2))



['$3.00']
[3.0]
3.0


In [362]:
import re
from statistics import mean

def calc_avg_price(v):
  if v.startswith('$'):
    nums_strings = re.findall(r"(?:[\£\$\€]{1}[,\d]+.?\d*)", "$3.00 - $5.75")
    nums_floats = [float(string.strip('$')) for string in nums_strings]
    return round(mean(nums_floats), 2)
  else:
    return np.nan



In [363]:
ipo_filings_dfs['Avg_price'] = ipo_filings_dfs['Price Range'].apply(lambda x:calc_avg_price(x))
ipo_filings_dfs.info()
ipo_filings_dfs.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  252 non-null    float64       
 5   Avg_price       258 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 15.4+ KB


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,4.38
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,4.38
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,4.38
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,4.38
6,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,4.38
7,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,4.38
8,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN,NaN
9,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN,NaN


In [364]:
ipo_filings_dfs['Shares_offered_value'] = ipo_filings_dfs['Shares Offered'] * ipo_filings_dfs['Avg_price']
ipo_filings_dfs.info()
ipo_filings_dfs.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           325 non-null    datetime64[ns]
 1   Symbol                325 non-null    object        
 2   Company Name          325 non-null    object        
 3   Price Range           325 non-null    object        
 4   Shares Offered        252 non-null    float64       
 5   Avg_price             258 non-null    float64       
 6   Shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 17.9+ KB


,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,4.38,16425000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,4.38,26280000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,4.38,21900000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,4.38,NaN
6,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,4.38,109500000.0
7,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,4.38,NaN
8,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN,NaN,NaN
9,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN,NaN,NaN


In [365]:
ipo_filings_dfs_2023 = ipo_filings_dfs[ipo_filings_dfs['Filing Date'].dt.year == 2023]
ipo_filings_dfs_2023_friday = ipo_filings_dfs_2023[ipo_filings_dfs_2023['Filing Date'].dt.day_name() == 'Friday']
ipo_filings_dfs_2023_friday.count()

Filing Date             32
Symbol                  32
Company Name            32
Price Range             32
Shares Offered          25
Avg_price               25
Shares_offered_value    25
dtype: int64

In [366]:
from numerize import numerize

sum_shares_offered_2023_friday = ipo_filings_dfs_2023_friday['Shares_offered_value'].sum()
sum_shares_offered_2023_friday_in_millions = int(round((sum_shares_offered_2023_friday/1000000), 0))
print(sum_shares_offered_2023_friday_in_millions)


250


#2) Question 2: IPOs "Fixed days hold" strategy

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).

In [367]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [368]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [369]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [370]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [371]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [372]:
stacked_ipos_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.65,19.38%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.15,-21.25%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$33.77,5.53%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.83,77.96%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.17,0.89%


In [373]:
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      218 non-null    object
 1   Symbol        218 non-null    object
 2   Company Name  218 non-null    object
 3   IPO Price     218 non-null    object
 4   Current       218 non-null    object
 5   Return        218 non-null    object
dtypes: object(6)
memory usage: 10.3+ KB


Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

In [374]:
from datetime import datetime

def convert_ipo_date_todate(v):
  d = datetime.strptime(v, '%b %d, %Y')
  return d

In [375]:
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'].apply(lambda x: convert_ipo_date_todate(x)))
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      218 non-null    datetime64[ns]
 1   Symbol        218 non-null    object        
 2   Company Name  218 non-null    object        
 3   IPO Price     218 non-null    object        
 4   Current       218 non-null    object        
 5   Return        218 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.3+ KB


In [376]:
ipo_filings_df_lt_mar2024 = stacked_ipos_df[stacked_ipos_df['IPO Date'] < datetime(2024,3,1)]
ipo_filings_df_lt_mar2024 = ipo_filings_df_lt_mar2024[~stacked_ipos_df['Symbol'].isin(['RYZB','PTHR'])]
ipo_filings_df_lt_mar2024.count()

<ipython-input-376-2efd0041ab6e>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ipo_filings_df_lt_mar2024 = ipo_filings_df_lt_mar2024[~stacked_ipos_df['Symbol'].isin(['RYZB','PTHR'])]


IPO Date        183
Symbol          183
Company Name    183
IPO Price       183
Current         183
Return          183
dtype: int64

In [377]:
ipo_filings_df_lt_mar2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.34,158.50%
34,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.13,-96.86%
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.84,-69.33%
37,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.07,-73.25%


In [378]:
ipo_filings_df_lt_mar2024.tail()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
213,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.19,-74.48%
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.48,-90.40%
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.61,150.73%
216,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%
217,2023-01-13,MGOL,MGO Global Inc.,$5.00,$0.22,-95.50%


In [379]:
# prompt: add 30 columns to ipo_filings_df_lt_mar2024

for i in range(30):
  ipo_filings_df_lt_mar2024[f'growth_future_{i+1}d'] = np.nan
ipo_filings_df_lt_mar2024.head()


,IPO Date,Symbol,Company Name,IPO Price,Current,Return,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.34,158.50%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.13,-96.86%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.84,-69.33%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.07,-73.25%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
# prompt: loop through each row in ipo_filings_df_lt_mar2024 and print the Symbol value

for symbol in ipo_filings_df_lt_mar2024['Symbol']:
  try:
    df = yf.download(tickers = symbol,
                     period = "max",
                     interval = "1d")
    for i in range(30):
      try:
        ipo_filings_df_lt_mar2024.loc[ipo_filings_df_lt_mar2024['Symbol'] == symbol, f'growth_future_{i+1}d'] = df['Adj Close'][i+1]-df['Adj Close'][i]
      except:
          ipo_filings_df_lt_mar2024.loc[ipo_filings_df_lt_mar2024['Symbol'] == symbol, f'growth_future_{i+1}d'] = np.nan
          continue
    #print(f"\nSuccess with {symbol}\n")
  except:
    print(f"\nError with {symbol}\n")
    continue
#errors: DYCQ, LEGT, PTHR

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [381]:
ipo_filings_df_lt_mar2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.34,158.50%,-2.140000,-0.970,-0.010,-0.220,...,0.100,1.250,1.810,-0.220,2.45,-1.660,-0.230,0.059999,1.000,-0.480
34,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.13,-96.86%,0.085000,-0.050,-0.290,0.100,...,-0.086,-0.062,-0.021,-0.098,-0.10,-0.101,-0.051,-0.051000,-0.022,0.023
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.16,1.60%,0.014999,0.005,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.84,-69.33%,0.000000,-0.020,0.020,-0.470,...,-0.090,-0.180,-0.080,-0.320,0.04,-0.060,0.270,-0.090000,-0.230,0.010
37,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.07,-73.25%,0.470000,-0.510,-0.475,-0.035,...,0.050,0.010,-0.460,-0.210,0.04,-0.119,0.129,0.230000,0.320,-0.600


In [382]:
ipo_filings_df_lt_mar2024.loc[ipo_filings_df_lt_mar2024['Symbol'] == 'MGOL']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
217,2023-01-13,MGOL,MGO Global Inc.,$5.00,$0.22,-95.50%,0.25,-1.2,0.23,-0.49,...,-0.06,0.14,0.2,-0.22,-0.135,0.075,0.19,-0.24,0.15,-0.14


In [383]:
#check for MOGL
ipo_max =0
for i in range(30):
    ipo_growth = ipo_filings_df_lt_mar2024.loc[ipo_filings_df_lt_mar2024['Symbol'] == 'MGOL', f'growth_future_{i+1}d'].iloc[0]
    if(ipo_growth == np.nan):
      break
    else:
      ipo_max = max(ipo_max, ipo_growth)

print(ipo_max)


0.28999996185302734


In [384]:
describe_ipo_filings_df = np.nan
describe_ipo_filings_df = ipo_filings_df_lt_mar2024.drop(['IPO Date'], axis=1).describe()
describe_ipo_filings_df

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,...,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,181.000000,181.000000,181.000000
mean,-19.536435,-1.035820,-1.193961,-0.934152,0.180673,0.592714,-1.003373,-0.530954,0.009730,-0.223810,...,-0.092946,-0.074126,-0.097155,-0.130635,-0.187948,-0.108019,-0.088402,0.031249,2.459494,0.718102
std,261.377162,7.927650,11.417681,11.168248,11.639156,10.550297,11.489130,5.318622,0.744869,4.038792,...,1.044701,0.884649,1.297779,1.896297,1.800543,1.358076,1.950200,1.362940,33.309100,10.580913
min,-3535.000000,-79.500000,-146.500000,-150.000000,-86.000000,-17.199997,-154.000000,-61.000000,-3.200001,-43.000000,...,-6.959999,-8.760000,-11.000000,-24.000000,-18.000000,-10.590000,-18.000000,-16.000000,-4.360001,-4.350002
25%,-0.530000,-0.245001,-0.289743,-0.225000,-0.175000,-0.205000,-0.210000,-0.187500,-0.130000,-0.115000,...,-0.125000,-0.080000,-0.178085,-0.140000,-0.137033,-0.170000,-0.119107,-0.050000,-0.109805,-0.140000
50%,0.000000,0.000000,-0.009999,0.000000,-0.010000,-0.008995,-0.009514,-0.009999,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.010000,0.000000,-0.010000,0.000000,0.010000,0.000000,0.000000
75%,0.082500,0.102331,0.075000,0.070000,0.074783,0.095000,0.098364,0.109500,0.100000,0.075000,...,0.090000,0.090000,0.090000,0.059500,0.084750,0.047171,0.068376,0.160000,0.099999,0.136999
max,72.000000,3.930820,4.889999,5.099999,131.000000,141.000000,3.119999,6.539999,3.490002,26.000000,...,4.682014,2.370000,11.000000,4.500000,2.500000,11.160000,15.150000,4.250000,448.000000,142.000000


In [385]:
describe_ipo_filings_df.loc['75%'].idxmax()

'growth_future_13d'

#3) Question 3: Is Growth Concentrated in the Largest Stocks?

In [386]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [387]:
stocks_largest_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(LARGEST_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     start=datetime(2014,1,1)-dt.timedelta(days=12),
                     end=datetime(2024,1,1),
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_largest_df.empty:
    stocks_largest_df = historyPrices
  else:
    stocks_largest_df = pd.concat([stocks_largest_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 AAPL



[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

30 ITC.NS


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [388]:
stocks_largest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80979 entries, 0 to 80978
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Open                          80979 non-null  float64
 1   High                          80979 non-null  float64
 2   Low                           80979 non-null  float64
 3   Close                         80979 non-null  float64
 4   Adj Close                     80979 non-null  float64
 5   Volume                        80979 non-null  int64  
 6   Ticker                        80979 non-null  object 
 7   Year                          80979 non-null  int32  
 8   Month                         80979 non-null  int32  
 9   Weekday                       80979 non-null  int32  
 10  Date                          80979 non-null  object 
 11  growth_1d                     80946 non-null  float64
 12  growth_3d                     80880 non-null  float64
 13  g

In [389]:
stocks_largest_df['Date'] = pd.to_datetime(stocks_largest_df['Date'])
stocks_largest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80979 entries, 0 to 80978
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Open                          80979 non-null  float64       
 1   High                          80979 non-null  float64       
 2   Low                           80979 non-null  float64       
 3   Close                         80979 non-null  float64       
 4   Adj Close                     80979 non-null  float64       
 5   Volume                        80979 non-null  int64         
 6   Ticker                        80979 non-null  object        
 7   Year                          80979 non-null  int32         
 8   Month                         80979 non-null  int32         
 9   Weekday                       80979 non-null  int32         
 10  Date                          80979 non-null  datetime64[ns]
 11  growth_1d                   

In [390]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'

In [391]:
stocks_largest_df['ticker_type'] = stocks_largest_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [392]:
stocks_largest_df['ticker_category'] = 'LARGEST'

In [393]:
stocks_largest_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type,ticker_category
80974,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,2023,12,4,...,2.245608,NaN,3436.02500,3330.172498,1,0.025130,2815.349848,0,INDIA,LARGEST
80975,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,2023,12,1,...,2.239815,NaN,3446.45000,3351.962500,1,0.008711,2827.964375,0,INDIA,LARGEST
80976,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,2023,12,2,...,2.298855,NaN,3466.61001,3376.662500,1,0.012655,2864.939949,0,INDIA,LARGEST
80977,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121,LT.NS,2023,12,3,...,2.294854,NaN,3478.43501,3398.377502,1,0.016899,2867.823712,0,INDIA,LARGEST
80978,3531.000000,3540.000000,3495.000000,3526.000000,3526.000000,968577,LT.NS,2023,12,4,...,2.221592,NaN,3487.72500,3419.217505,1,0.012762,2866.029127,0,INDIA,LARGEST


In [394]:
# count of observations between US-EU-INDIA stocks
stocks_largest_df.ticker_type.value_counts()

ticker_type
EU       27950
US       27753
INDIA    25276
Name: count, dtype: int64

In [395]:
# unique tickers
stocks_largest_df.Ticker.nunique()

33

In [396]:
# count of observations by stock
stocks_largest_df.Ticker.value_counts()

Ticker
RMS.PA           2567
CDI.PA           2567
OR.PA            2567
MC.PA            2567
SIE.DE           2544
MSFT             2523
INFY             2523
IBN              2523
HDB              2523
IDEXY            2523
TTE              2523
ACN              2523
AAPL             2523
SAP              2523
ASML             2523
GOOG             2523
JPM              2523
V                2523
AVGO             2523
LLY              2523
BRK-B            2523
META             2523
NVO              2523
AMZN             2523
NVDA             2523
HINDUNILVR.NS    2472
ITC.NS           2472
TCS.NS           2472
SBIN.NS          2472
BHARTIARTL.NS    2472
RELIANCE.NS      2472
LT.NS            2472
LICI.NS           403
Name: count, dtype: int64

In [397]:
stocks_largest_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,2013-12-20,2023-12-29
ACN,EU,2013-12-20,2023-12-29
AMZN,US,2013-12-20,2023-12-29
ASML,EU,2013-12-20,2023-12-29
AVGO,US,2013-12-20,2023-12-29
BHARTIARTL.NS,INDIA,2013-12-20,2023-12-29
BRK-B,US,2013-12-20,2023-12-29
CDI.PA,EU,2013-12-20,2023-12-29
GOOG,US,2013-12-20,2023-12-29


In [398]:
average_growth_7d = stocks_largest_df[stocks_largest_df['Date'] == datetime(2014, 1, 1)]['growth_7d'].mean(skipna=True)
print(f"The average growth_7d for LARGEST stocks on {datetime(2014, 1, 1)} is {average_growth_7d}")

The average growth_7d for LARGEST stocks on 2014-01-01 00:00:00 is 1.0117973097333546


In [399]:
stocks_largest_flatten_df = pd.pivot_table(stocks_largest_df, values='growth_7d', index=['Date'], aggfunc="mean")
stocks_largest_flatten_df.head()

,growth_7d
Date,
2014-01-01,1.011797
2014-01-02,1.002964
2014-01-03,0.997668
2014-01-06,0.993423
2014-01-07,0.992509


In [400]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [401]:
stocks_large_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(LARGE_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     start=datetime(2014,1,1)-dt.timedelta(days=12),
                     end=datetime(2024,1,1),
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_large_df.empty:
    stocks_large_df = historyPrices
  else:
    stocks_large_df = pd.concat([stocks_large_df, historyPrices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed

0 PRX.AS



[*********************100%%**********************]  1 of 1 completed

1 CDI.PA



[*********************100%%**********************]  1 of 1 completed

...

31 NTPC.NS



[*********************100%%**********************]  1 of 1 completed

32 KOTAKBANK.NS



[*********************100%%**********************]  1 of 1 completed

33 TITAN.NS


In [402]:
stocks_large_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84065 entries, 0 to 84064
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Open                          84065 non-null  float64
 1   High                          84065 non-null  float64
 2   Low                           84065 non-null  float64
 3   Close                         84065 non-null  float64
 4   Adj Close                     84065 non-null  float64
 5   Volume                        84065 non-null  int64  
 6   Ticker                        84065 non-null  object 
 7   Year                          84065 non-null  int32  
 8   Month                         84065 non-null  int32  
 9   Weekday                       84065 non-null  int32  
 10  Date                          84065 non-null  object 
 11  growth_1d                     84031 non-null  float64
 12  growth_3d                     83963 non-null  float64
 13  g

In [403]:
stocks_large_df['Date'] = pd.to_datetime(stocks_large_df['Date'])
stocks_large_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84065 entries, 0 to 84064
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Open                          84065 non-null  float64       
 1   High                          84065 non-null  float64       
 2   Low                           84065 non-null  float64       
 3   Close                         84065 non-null  float64       
 4   Adj Close                     84065 non-null  float64       
 5   Volume                        84065 non-null  int64         
 6   Ticker                        84065 non-null  object        
 7   Year                          84065 non-null  int32         
 8   Month                         84065 non-null  int32         
 9   Weekday                       84065 non-null  int32         
 10  Date                          84065 non-null  datetime64[ns]
 11  growth_1d                   

In [404]:
stocks_large_df['ticker_type'] = stocks_large_df.Ticker.apply(lambda x:get_ticker_type(x, NEW_US, NEW_EU, NEW_INDIA))

In [405]:
stocks_large_df['ticker_category'] = 'LARGE'

In [406]:
stocks_large_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future,ticker_type,ticker_category
84060,3580.000000,3638.449951,3560.550049,3627.350098,3627.350098,777099,TITAN.NS,2023,12,4,...,1.876465,NaN,3598.770020,3551.854993,1,0.021476,1964.038283,0,INDIA,LARGE
84061,3635.000000,3665.000000,3623.449951,3656.699951,3656.699951,526101,TITAN.NS,2023,12,1,...,1.875937,NaN,3601.390015,3564.842493,1,0.011363,1937.960346,0,INDIA,LARGE
84062,3668.000000,3695.000000,3645.000000,3689.250000,3689.250000,666625,TITAN.NS,2023,12,2,...,1.900625,NaN,3613.175024,3577.184998,1,0.013553,1885.710504,0,INDIA,LARGE
84063,3699.899902,3737.000000,3680.699951,3715.100098,3715.100098,1033648,TITAN.NS,2023,12,3,...,1.857576,NaN,3624.450024,3591.475000,1,0.015154,1834.906372,0,INDIA,LARGE
84064,3715.100098,3715.100098,3660.250000,3675.449951,3675.449951,709614,TITAN.NS,2023,12,4,...,1.739320,NaN,3632.855029,3600.717493,1,0.014923,1757.415488,0,INDIA,LARGE


In [407]:
# count of observations between US-EU-INDIA stocks
stocks_large_df.ticker_type.value_counts()

ticker_type
EU       29120
US       27753
INDIA    27192
Name: count, dtype: int64

In [408]:
# unique tickers
stocks_large_df.Ticker.nunique()

33

In [409]:
# count of observations by stock
stocks_large_df.Ticker.value_counts()

Ticker
ADANIENT.NS      4944
CDI.PA           2567
AIR.PA           2567
SU.PA            2567
AI.PA            2567
EL.PA            2567
DTE.DE           2544
ALV.DE           2544
MA               2523
ORCL             2523
COST             2523
HD               2523
MRK              2523
JNJ              2523
PG               2523
WMT              2523
UNH              2523
XOM              2523
TSLA             2523
MDT              2523
BUD              2523
SNY              2523
ETN              2523
SUNPHARMA.NS     2472
KOTAKBANK.NS     2472
NTPC.NS          2472
ONGC.NS          2472
TITAN.NS         2472
TATAMOTORS.NS    2472
HCLTECH.NS       2472
MARUTI.NS        2472
BAJFINANCE.NS    2472
PRX.AS           1105
Name: count, dtype: int64

In [410]:
stocks_large_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
ADANIENT.NS,INDIA,2013-12-20,2023-12-29
AI.PA,EU,2013-12-20,2023-12-29
AIR.PA,EU,2013-12-20,2023-12-29
ALV.DE,EU,2013-12-20,2023-12-29
BAJFINANCE.NS,INDIA,2013-12-20,2023-12-29
BUD,EU,2013-12-20,2023-12-29
CDI.PA,EU,2013-12-20,2023-12-29
COST,US,2013-12-20,2023-12-29
DTE.DE,EU,2013-12-20,2023-12-29


In [411]:
# prompt: calculate average of stocks_df2['growth_7d'] where stocks_df2['Date'] == datetime(2014, 1, 1)

average_growth_7d = stocks_large_df[stocks_large_df['Date'] == datetime(2014, 1, 1)]['growth_7d'].mean(skipna=True)
print(f"The average growth_7d for LARGE stocks on {datetime(2014, 1, 1)} is {average_growth_7d}")



The average growth_7d for LARGE stocks on 2014-01-01 00:00:00 is 1.0116842806709583


In [412]:
stocks_large_flatten_df = pd.pivot_table(stocks_large_df, values='growth_7d', index=['Date'], aggfunc="mean")
stocks_large_flatten_df.head()

,growth_7d
Date,
2014-01-01,1.011684
2014-01-02,0.997706
2014-01-03,0.999534
2014-01-06,0.995876
2014-01-07,0.991592


In [413]:
#stack the LARGE and LARGEST dataframes
stacked_stocks_large_largest_df = pd.concat([stocks_largest_df, stocks_large_df], ignore_index=True)
stacked_stocks_large_largest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165044 entries, 0 to 165043
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Open                          165044 non-null  float64       
 1   High                          165044 non-null  float64       
 2   Low                           165044 non-null  float64       
 3   Close                         165044 non-null  float64       
 4   Adj Close                     165044 non-null  float64       
 5   Volume                        165044 non-null  int64         
 6   Ticker                        165044 non-null  object        
 7   Year                          165044 non-null  int32         
 8   Month                         165044 non-null  int32         
 9   Weekday                       165044 non-null  int32         
 10  Date                          165044 non-null  datetime64[ns]
 11  growth_1d    

In [414]:
stacked_stocks_large_largest_flatten_df = pd.pivot_table(stacked_stocks_large_largest_df, values='growth_7d', index='Date', columns='ticker_category', aggfunc="mean")
stacked_stocks_large_largest_flatten_df.tail()

ticker_category,LARGE,LARGEST
Date,,
2023-12-22,1.003014,1.013788
2023-12-26,1.013505,1.014900
2023-12-27,1.002167,1.014155
2023-12-28,0.998583,1.008533
2023-12-29,0.999313,1.005337


In [415]:
filter_l_outperforms = (stacked_stocks_large_largest_flatten_df['LARGE'] > stacked_stocks_large_largest_flatten_df['LARGEST'])
stacked_stocks_large_largest_flatten_df['L_outperforms'] = filter_l_outperforms
stacked_stocks_large_largest_flatten_df

ticker_category,LARGE,LARGEST,L_outperforms
Date,,,
2014-01-01,1.011684,1.011797,False
2014-01-02,0.997706,1.002964,False
2014-01-03,0.999534,0.997668,True
2014-01-06,0.995876,0.993423,True
2014-01-07,0.991592,0.992509,False
...,...,...,...
2023-12-22,1.003014,1.013788,False
2023-12-26,1.013505,1.014900,False
2023-12-27,1.002167,1.014155,False


In [416]:
stacked_stocks_large_largest_flatten_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2595 entries, 2014-01-01 to 2023-12-29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LARGE          2595 non-null   float64
 1   LARGEST        2595 non-null   float64
 2   L_outperforms  2595 non-null   bool   
dtypes: bool(1), float64(2)
memory usage: 63.4 KB


In [417]:
# prompt: get count of stacked_stocks_large_largest_flatten_df where L_outperforms is true

l_outperforms_count = stacked_stocks_large_largest_flatten_df[stacked_stocks_large_largest_flatten_df['L_outperforms'] == bool(True)].count()['L_outperforms']
print(f"LARGE outperformed LARGEST {l_outperforms_count} times")
ls_outperforms_count = stacked_stocks_large_largest_flatten_df[stacked_stocks_large_largest_flatten_df['L_outperforms'] == bool(False)].count()['L_outperforms']
print(f"LARGEST outperformed LARGE {ls_outperforms_count} times")


LARGE outperformed LARGEST 1231 times
LARGEST outperformed LARGE 1364 times


In [418]:
total_trading_days = len(stacked_stocks_large_largest_flatten_df.index)
print(total_trading_days)

2595


In [419]:
#Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value).
print(f"The share of days when Large Stocks outperform the Largest stocks is {(l_outperforms_count/total_trading_days):.0%}")

The share of days when Large Stocks outperform the Largest stocks is 47%


#4) Question 4: Trying Another Technical Indicators strategy

What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

In [420]:
q4_df = np.nan;
q4_df = pd.read_parquet('/content/sample_data/stocks_df_combined_trunc_2014_2023.parquet.brotli')
q4_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 105.4+ MB


In [421]:
q4_df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [422]:
#add cci_over_threshold column
q4_df['cci_over_threshold'] = (q4_df['cci'] >= 200)
q4_df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,cci_over_threshold
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN,False
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN,False
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN,False
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN,False
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN,False


In [423]:
# unique tickers
q4_df.Ticker.nunique()

33

In [424]:
q4_df.groupby(['Ticker','ticker_type']).Date.agg(['min','max'])

,,min,max
Ticker,ticker_type,,
AAPL,US,2014-01-02,2023-12-29
ACN,EU,2014-01-02,2023-12-29
AMZN,US,2014-01-02,2023-12-29
ASML,EU,2014-01-02,2023-12-29
AVGO,US,2014-01-02,2023-12-29
BHARTIARTL.NS,INDIA,2014-01-01,2023-12-29
BRK-B,US,2014-01-02,2023-12-29
CDI.PA,EU,2014-01-02,2023-12-29
GOOG,US,2014-01-02,2023-12-29


In [425]:
#why did he say 460? there are 521 Fridays in the dataset
#there is a Weekday column in the dataset. 4 would indicate a FRI
def get_week_of_date(date):
    # %U calculates the week number starting from Sunday
    # %W calculates the week number starting from Monday
    # Replace %U with %W if you want to start counting weeks from Monday
    week_number = int(date.strftime("%W"))
    return week_number

# Example usage:
date = dt.datetime(2024, 5, 5)  # Your desired date
week_number = get_week_of_date(date)
print("Week number of", date, "is:", week_number)

Week number of 2024-05-05 00:00:00 is: 18


In [426]:
#confirm CCI went over threshold during 460 weeks of the dataset > i got 443 :(

#add a Week column to the dataset
q4_df['week_of_date'] = q4_df['Date'].apply(get_week_of_date)
q4_df.tail()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,cci_over_threshold,week_of_date
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201,False,51
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055,False,52
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696,False,52
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301,False,52
5342,3531.000000,3540.000000,3495.000000,3526.000000,3526.000000,968577.0,LT.NS,2023,2023-12-01,4,...,0.925850,0.720539,0.987603,0.990099,0.956850,1.112020,1.561092,2.529656,False,52


In [427]:
#q4_cci_over_threshold_grouped_by_week_df = q4_df.groupby(['week_of_date', 'Ticker']).cci_over_threshold.sum()
q4_grouped_by_week_df = q4_df.groupby(['Year','week_of_date']).cci_over_threshold.sum().reset_index()
#print(len(q4_cci_over_threshold_grouped_by_week_df.index))
q4_grouped_by_week_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Year                527 non-null    int32
 1   week_of_date        527 non-null    int64
 2   cci_over_threshold  527 non-null    int64
dtypes: int32(1), int64(2)
memory usage: 10.4 KB


In [428]:
q4_grouped_by_week_df.head()

,Year,week_of_date,cci_over_threshold
0,2014,0,0
1,2014,1,1
2,2014,2,7
3,2014,3,2
4,2014,4,3


In [429]:
#validate a row
cci_over_threshold_2014_week_2_count = q4_df[(q4_df['cci_over_threshold'] == bool(True)) & (q4_df['Year'] == 2014) & (q4_df['week_of_date'] == 2)].count()['cci_over_threshold']
print(f"CCI was over threshold {cci_over_threshold_2014_week_2_count} times in 2014 week 2")

CCI was over threshold 7 times in 2014 week 2


In [430]:
# prompt: get count of records where cci_over_threshold is greater than 0

total_weeks_cci_over_threshold = q4_grouped_by_week_df[q4_grouped_by_week_df['cci_over_threshold'] > 0].count()['cci_over_threshold']
print(f"CCI was over threshold in {total_weeks_cci_over_threshold} weeks of the period")

CCI was over threshold in 443 weeks of the period


In [431]:
#...scratch that...let's try a different approach

#get all records where CCI goes over threshold on a FRI
#BUT...what if CCI goes over threshold earlier in the week...this will not catch it
filtered_cci_over_threshold_q4_df = q4_df[(q4_df['Weekday'] == 4) & (q4_df['cci_over_threshold'] == bool(True))]
filtered_cci_over_threshold_q4_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460 entries, 7113 to 5323
Columns: 204 entries, Open to week_of_date
dtypes: bool(1), datetime64[ns](3), float64(128), int32(64), int64(6), object(2)
memory usage: 618.6+ KB


In [432]:
#validate the growth_future_5d is accurate on a sample

q4_df['Date'].dtype
q4_df[(q4_df['Ticker']=='MSFT') & (q4_df['Date']==np.datetime64('2014-06-06T00:00:00'))]

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,cci_over_threshold,week_of_date
7118,41.48,41.66,41.240002,41.48,35.373264,24060500.0,MSFT,2014,2014-06-01,4,...,1.017996,0.988802,NaN,NaN,NaN,NaN,NaN,NaN,False,22


In [433]:
#let's assume we can only by the # of shares we can afford with the $1000.  That is, if each share is greater than $1000, we can't invest

gross_profit = 0

for index, row in filtered_cci_over_threshold_q4_df.iterrows():
    # Access row values using row[column_name] or row[column_index]
    ticker = row['Ticker']
    date = row['Date']
    adj_close_price = row['Adj Close_x']
    growth_future_5d = row['growth_future_5d']
    #print(f"Ticker: {ticker}, Date: {date}, Adj Close Price: {adj_close_price}, Growth Future 5d: {growth_future_5d}")
    stocks_purchased = 1000 // adj_close_price
    invested_amt = stocks_purchased * adj_close_price
    sold_amt = stocks_purchased * (adj_close_price * growth_future_5d)
    gross_profit += (sold_amt - invested_amt)
    #note: in some instances the CCI > 200 but the stock cost > 1000, so did not invest anything in that scenario
    #print(f"Stocks Purchased: {stocks_purchased}, Invested Amount: {invested_amt}, Sold Amount: {sold_amt}")

print(f"Gross Profit: {gross_profit} rounded to nearest thousand is {math.ceil(gross_profit / 1000)} THOUSAND")


Gross Profit: 809.6596460342399 rounded to nearest thousand is 1 THOUSAND


In [434]:
filtered_cci_over_threshold_gt_1k_q4_df = filtered_cci_over_threshold_q4_df[(filtered_cci_over_threshold_q4_df['Adj Close_x'] <= 1000)]
filtered_cci_over_threshold_gt_1k_q4_df.info()
trades_count = len(filtered_cci_over_threshold_gt_1k_q4_df.index)
print(f"Trades count: {trades_count}")

<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 7113 to 3466
Columns: 204 entries, Open to week_of_date
dtypes: bool(1), datetime64[ns](3), float64(128), int32(64), int64(6), object(2)
memory usage: 556.7+ KB
Trades count: 414


Answer 1 - where limited to amount willing invest ($1000) is fixed on share price - NO PARTIAL INVESTMENTS.  

Total profit rounded to nearest thousand is 1.  

Once factor in fees from https://www.degiro.ie/fees/calculator where amount of transation equals 1000 and transactions per year equals 414 yielding €1,989.70 in fees - this WOULD NOT be a profitable investment strategy as that is more than the calculated gross profit.

In [435]:
#instead let's just invest $1000 each time (assume we can invest in fractional shares)

gross_profit = 0

for index, row in filtered_cci_over_threshold_q4_df.iterrows():
    # Access row values using row[column_name] or row[column_index]
    ticker = row['Ticker']
    date = row['Date']
    adj_close_price = row['Adj Close_x']
    growth_future_5d = row['growth_future_5d']
    #print(f"Ticker: {ticker}, Date: {date}, Adj Close Price: {adj_close_price}, Growth Future 5d: {growth_future_5d}")
    invested_amt = 1000
    sold_amt = (invested_amt * growth_future_5d)
    gross_profit += (sold_amt - invested_amt)
    #note: in some instances the CCI > 200 but the stock cost > 1000, so did not invest anything in that scenario
    #print(f"Stocks Purchased: {stocks_purchased}, Invested Amount: {invested_amt}, Sold Amount: {sold_amt}")

print(f"Gross Profit: {gross_profit} rounded to nearest thousand is {math.ceil(gross_profit / 1000)} THOUSAND")


Gross Profit: 1048.262891817731 rounded to nearest thousand is 2 THOUSAND


[FINAL] Answer 2 - PARTIAL INVESTMENTS ALLOWED.  Always invest the $1000

Total profit rounded to nearest thousand is 2.  

Once factor in fees from https://www.degiro.ie/fees/calculator where amount of transation equals 1000 and transactions per year equals 460 yielding €2,210.50 in fees - this still WOULD NOT be a profitable investment strategy as that is more than the calculated gross profit.